In [28]:
import numpy as np
import pandas as pd
import scipy.io as io
import os

In [29]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

torch.cuda.is_available()

True

In [37]:
import timm
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler
from sklearn.model_selection import train_test_split

In [30]:
import wandb
WANDB_NOTEBOOK_NAME = 'Train_Analysis_VRB_wandb_sweep.ipynb'
wandb.login()

True

In [31]:
config={
    'batch_size':64,
    'lr':0.0001,
    'epochs':5
}

sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
    'parameters': 
    {
        'batch_size': {'values': [32]},
        'epochs': {'values': [5]},
        'lr': {'values': [0.001, 0.0001]}
     }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project='VRB-Pytorch-test')


Create sweep with ID: 87ggx27m
Sweep URL: https://wandb.ai/xiaosuhu86/VRB-Pytorch-test/sweeps/87ggx27m


In [32]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [33]:
class FNIRS_Dataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.targets = self.img_labels['Label']

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 4])
        image = np.float32(io.loadmat(img_path)['fnirsimg']).reshape(3,21,45)
        label = self.img_labels.iloc[idx, 5]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [34]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return loss

def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()     # Optional when not using Model Specific layer

    total_correct = 0
    total_instances = 0

    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)
        classifications = torch.argmax(model(X), dim=1)
        correct_predictions = sum(classifications==y).item()

        total_correct+=correct_predictions
        total_instances+=len(y)
            
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"val_loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    acc = total_correct/total_instances
    print(f"val_acc: {acc:>7f}")

    return loss, acc

In [36]:
# Load the data with imbalanced weights
train_data=FNIRS_Dataset(
    '../Label_VRB.csv',
    '../VRBdata/'
)

total_targets=torch.asarray(train_data.targets)
train_idx, valid_idx= train_test_split(
    np.arange(len(total_targets)), test_size=0.2, random_state=42, shuffle=True, stratify=total_targets)

train_set = torch.utils.data.Subset(train_data, train_idx)
# val_set = torch.utils.data.Subset(train_data, valid_idx)

train_sample_count = torch.tensor(
    [(total_targets[train_idx] == t).sum() for t in torch.unique(total_targets, sorted=True)])
train_weight = 1. / train_sample_count.float()
train_sample_weight = torch.tensor([train_weight[t] for t in total_targets[train_idx]])

#Creating PT data samplers
train_sampler = WeightedRandomSampler(train_sample_weight, len(train_sample_weight))
valid_sampler = SubsetRandomSampler(valid_idx)

def main():
    run = wandb.init()

    batch_size = wandb.config.batch_size

    # Create data loaders:
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                            sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                                    sampler=valid_sampler)

    # Model defining
    model = timm.create_model('legacy_seresnet18', num_classes=2, pretrained=False)
    model=model.to(device)
    
    # Other learning parameters
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr)
    epochs = wandb.config.epochs

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc = validate(validation_loader, model, loss_fn)

        wandb.log({
        'epoch': t+1, 
        #'train_acc': train_acc,
        'train_loss': train_loss, 
        'val_acc': val_acc, 
        'val_loss': val_loss
        })

wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: w3bxo0zt with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	lr: 0.004591429675591624
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------


Run w3bxo0zt errored: FileNotFoundError(2, 'No such file or directory')
wandb: ERROR Run w3bxo0zt errored: FileNotFoundError(2, 'No such file or directory')
wandb: Agent Starting Run: e56rf8ai with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	lr: 0.006395440690108898
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------


Run e56rf8ai errored: FileNotFoundError(2, 'No such file or directory')
wandb: ERROR Run e56rf8ai errored: FileNotFoundError(2, 'No such file or directory')
wandb: Agent Starting Run: 2q14eqxc with config:
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	lr: 0.001617613911461978
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1
-------------------------------


Run 2q14eqxc errored: FileNotFoundError(2, 'No such file or directory')
wandb: ERROR Run 2q14eqxc errored: FileNotFoundError(2, 'No such file or directory')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true
